In [20]:
from opentrons import protocol_api, types
from opentrons import containers
import time
import csv

In [21]:

def run(protocol:protocol_api.ProtocolContext):
    
    """
    Creates 2 tip racks,1x 15 mL reservoit, 1x 24 position tube, 1x 96 position
    
    Transfer 100 uL 
    """
    # Load Tips
    tip_20 = protocol.load_labware('opentrons_96_filtertiprack_20ul',1)
    tip_200 = protocol.load_labware('opentrons_96_filtertiprack_200ul',2)
    
    #Load Liquid reservoirs
    reservoir = protocol.load_labware('nest_12_reservoir_15ml', 4)
    block_96well = protocol.load_labware('opentrons_96_aluminumblock_generic_pcr_strip_200ul',5)
    
    #Load Pipettes
    p20 = protocol.load_instrument('p20_single_gen2', 'left', tip_racks = [tip_20])
    p300_multi = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks = [tip_200])

    #Load Modules
    thermal = protocol.load_module("temperature module gen2", 9)
    thermal_wells = thermal.load_labware('opentrons_24_aluminumblock_generic_2ml_screwcap',label="Enzyme Reactions")
    
    # Set temperature and wait
    thermal.set_temperature(37)
    
    #Designate Liquids
    time_points = [1,5,10,20,30,60,120,240] # When to take time point measurements
    enzyme_reactions = thermal_wells.wells()[0:3]
    acid_solution = reservoir.wells()[0]
    neutral_buffer = reservoir.wells()[1]
    protocol.pause(f"Please check: Acid at {acid_solution}, buffer at {neutral_buffer}, " \
                   f"and samples from {block_96well.wells()[0]} to {block_96well.wells()[len(time_points)*len(enzyme_reactions)]} " \
                   "Press Resume to continue")

    protocol.pause("Thermal is at {:.2f}. Please load samples to thermal block: {}. Press Resume to continue".format(thermal.temperature,enzyme_reactions))
    start_time = time.time()
    

    
    # Be prepared to record information 
    data = {'time':[],'sample_location':[], 'enzyme_location':[],'time_of_transfer':[]}
    file_out = open('enzyme_reaction.csv','a', newline='')
    data_out = csv.DictWriter(file_out, data.keys())
    data_out.writeheader()
    
    
    sample_idx = 0
    samples=[]
    for idx, time_delay in enumerate(time_points):
        protocol.delay(time_delay*60)
        sample_wells = block_96well.wells()[sample_idx:sample_idx+len(enzyme_reactions)]
        sample_idx +=1
        p20.transfer(9, acid_solution, sample_wells)
        samples.extend(sample_wells)
        for enzyme_reaction, sample_well in zip(enzyme_reactions, sample_wells):
            p20.transfer(1, [enzyme_reaction],[sample_well], new_tip = 'always', mix_after=(5,10))
            transfer_time = time.time() - start_time

            
            # Store data in dict and write to csv
            data['time'].append(time_delay)
            data['time_of_transfer'].append(transfer_time)
            data['sample_location'].append(sample_well)
            data['enzyme_location'].append(enzyme_reaction)
            data_out.writerow({'time':time_delay,'enzyme_location':enzyme_reaction,
                           'time_of_transfer':transfer_time,'sample_location':sample_well})
    
    file_out.close()


    #Fill the PCR tubes with samples (create the first sample, then serially dilute from that)
    print(neutral_buffer, samples[0])
    #p300_multi.move_to(protocol.deck.position_for('5').move(types.Point(z=100)))
    #p300_multi.transfer(190, [neutral_buffer], samples[0], mix_after=(5,150),new_tip='always', blow_out=True)
    
    return (neutral_buffer, samples)

In [22]:
from opentrons import simulate

protocol = simulate.get_protocol_api('2.4')
metadata = {'apiLevel': '2.3'}
x = run(protocol)

C:\Users\bvp22\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\bvp22\.opentrons\robot_settings.json not found. Loading defaults


A2 of NEST 12 Well Reservoir 15 mL on 4 A1 of Opentrons 96 Well Aluminum Block with Generic PCR Strip 200 µL on 5
